In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch('http://localhost:9200')

In [4]:
es_client.info()

ObjectApiResponse({'name': '7a355ab1219f', 'cluster_name': 'docker-cluster', 'cluster_uuid': '6az5-82rRtSc12hnU2tM_A', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:03<00:00, 274.69it/s]


In [9]:
query = 'How do I execute a command on a Kubernetes pod?'

In [10]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

results = es_client.search(index=index_name, body=search_query)

In [11]:
results['hits']['hits'][0]['_score']

55.1689

In [12]:
query = 'How do copy a file to a Docker container?' 

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

results = es_client.search(index=index_name, body=search_query)

In [13]:
results['hits']['hits']


[{'_index': 'course-questions',
  '_id': 'nnSxEZgBEmHUmztU5UNg',
  '_score': 73.38676,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'vXSxEZgBEmHUmztU5UPO',
  '_score': 66.688705,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

In [14]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [15]:
context_pieces = []

for hit in results['hits']['hits']:
    doc = hit['_source']
    context_piece = context_template.format(**doc)
    context_pieces.append(context_piece)

context = '\n\n'.join(context_pieces)

In [16]:
prompt = prompt_template.format(question=query, context=context)

In [17]:
len(prompt)

1446

In [18]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [19]:
import tiktoken

In [20]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [21]:
len(encoding.encode(prompt))

320

In [22]:
encoding.decode_single_token_bytes(4122)

b' based'

In [23]:
tokens = encoding.encode(prompt)[:10]
tokens

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122]

In [26]:
from google import genai

client = genai.Client()

response =  client.models.generate_content(
    model="gemini-2.5-flash",
    contents = prompt
)
print(response.text)

You can copy files from your local machine into a Docker container using the `docker cp` command. The basic syntax is:

`docker cp /path/to/local/file_or_directory container_id:/path/in/container`
